In [205]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from random import shuffle


pd.set_option('display.max_colwidth', -1)

In [288]:
train=pd.read_csv('train.csv')
#train.set_index('Webpage_id',inplace=True)
test=pd.read_csv('test_nvPHrOx.csv')
#test.set_index('Webpage_id',inplace=True)
submission=pd.read_csv('sample_submission_poy1UIu.csv')

In [207]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53447 entries, 1 to 79345
Data columns (total 3 columns):
Domain    53447 non-null object
Url       53447 non-null object
Tag       53447 non-null object
dtypes: object(3)
memory usage: 1.6+ MB


In [113]:
train.head(100)

,Webpage_id,Domain,Url,Tag
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfidera-gilenya-and-aubagio-s-3-way-battle-for-ms-share-about-to-get-more-interesting,news
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipped-to-weather-storm-u-s-diabetes-market-ceo-says,news
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exec-departs-troubled-endo-and-time-it-s-for-another-drugmaker,news
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-biosim-specialist-celltrion-it-wouldn-t-say-no,news
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-marissa-tomei-partners-allergan-restasis-to-drive-dry-eye-awareness,news
5,6,www.fiercepharma.com,http://www.fiercepharma.com/pharma/celgene-s-otezla-wins-nice-draft-backing-psoriatic-arthritis,news
6,7,www.fiercepharma.com,http://www.fiercepharma.com/pharma/mylan-takes-punching-bag-role-again-as-clinton-aims-to-shame-bad-actors,news
7,8,www.fiercepharma.com,http://www.fiercepharma.com/marketing/failed-eylea-combo-trial-throws-a-wrench-regeneron-s-expansion-effort,news
8,9,www.fiercepharma.com,http://www.fiercepharma.com/pharma/should-gsk-investors-be-disgruntled-about-its-insider-ceo-pick-if-they-want-big-changes-yes,news
9,10,www.fiercepharma.com,http://www.fiercepharma.com/pharma/drug-pricing-uproar-could-grow-competition-gpha-head-says,news


In [261]:
a=train[train.index==79300].Url.iloc[0]
a

'https://findadoc.ariahealth.org/details/189/anthony-abbruzzi-family_medicine-philadelphia'

In [227]:
pattern=r'//.*'
b=re.findall(pattern,a)
c=b[0][2:]
d=re.split('\.|/|-|_',c)
e=''
urls=[]
for i in d:
    e+=str(i).lower()
    e+=' '
e = ''.join([i for i in e if not i.isdigit()])
f=''
for i in e:
    if i.isalpha() or i==' ':
        #print(i)
        f+=i
    else:
        f+=' '

urls.append(f)

In [242]:
f

'www intelsatgeneral com news releases intelsat epicng delivering on the promise of high throughput with high performance  '

In [257]:
from nltk.collocations import BigramCollocationFinder

finder = BigramCollocationFinder.from_words(f)
finder.apply_freq_filter(5)
bigram_model = finder.ngram_fd.items()
bigram_model = sorted(finder.ngram_fd.items(), key = lambda item : item[1], reverse = True)

In [258]:
print(bigram_model)

[]


In [259]:
for i in finder.ngram_fd.items():
    print(i)

In [57]:
#for creating ngrams

#convert all to lower case -------- done
#all digits to be removed ----------done
#Punctuation marks and special characters were removed ----------done (no improvement)
#All the sentences were concatenated with space in between
#Series of contiguous white spaces were replaced by single space

In [308]:
def preprocess(url,title):
    
    pattern=r'//.*'
    urls=[]
    for i in url:
        b=re.findall(pattern,i)
        c=b[0][2:]
        d=re.split('\.|/|-|_',c)
        e=''
        for i in d:
            e+=str(i).lower()
            e+=' '
        e = ''.join([i for i in e if not i.isdigit()])
        f=''
        for i in e:
            if i.isalpha() or i==' ':
                f+=i
            else:
                f+=' '
        urls.append(f)
        
    titles=[]
    for e in title:    
        e = ''.join([i for i in e if not i.isdigit()])
        f=''
        for i in e:
            if i.isalpha() or i==' ':
                #print(i)
                f+=i
            else:
                f+=' '
        f = re.sub(' +',' ',f) # replace series of spaces with single space
        titles.append(f)
    
    data=[]
    for i in range(len(title)):
        s=titles[i]+' '+urls[i]
        data.append(str(s))
        
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(urls)
    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

    
    
    
    return(data)

# MultinomialNB

In [40]:
urls=preprocess(train.Url)
target=train.Tag.values

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(urls, target)

In [41]:
predicted = text_clf.predict(urls)
np.mean(predicted == target)

0.9235317230153236

In [42]:
test_urls=preprocess(test.Url)
predicted = text_clf.predict(test_urls)

In [43]:
predicted.shape

(25787,)

In [49]:
submission.drop('Tag',axis=1,inplace=True)

In [51]:
submission['Tag']=predicted

In [52]:
submission.head()

,Webpage_id,Tag
0,31,clinicalTrials
1,32,others
2,33,others
3,34,others
4,35,others


In [53]:
submission.to_csv('result.csv', index=False)

# SVM

In [309]:
from sklearn.linear_model import SGDClassifier
urls=preprocess(train.Url,new_data.Html)
target=train.Tag.values

text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(urls, target)
predicted_svm = text_clf_svm.predict(urls)
np.mean(predicted_svm == target)

C:\Users\Nishant\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8842965928864108

In [ ]:
#0.8949613635938406

In [310]:
test_urls=preprocess(test.Url,new_data_test.Html)
predicted = text_clf_svm.predict(test_urls)

In [311]:
predicted.shape

(25787,)

In [312]:
submission.drop('Tag',axis=1,inplace=True)
submission['Tag']=predicted
submission.to_csv('result_svm_with_new_data.csv', index=False)

In [313]:
submission.head()

,Webpage_id,Tag
0,31,clinicalTrials
1,32,clinicalTrials
2,33,clinicalTrials
3,34,clinicalTrials
4,35,clinicalTrials


# Train Val split

In [5]:
53447/3

17815.666666666668

In [6]:
17815*2

35630

In [7]:
35630/17815

2.0

In [27]:
train.head()

,Webpage_id,Domain,Url,Tag
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfidera-gilenya-and-aubagio-s-3-way-battle-for-ms-share-about-to-get-more-interesting,news
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipped-to-weather-storm-u-s-diabetes-market-ceo-says,news
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exec-departs-troubled-endo-and-time-it-s-for-another-drugmaker,news
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-biosim-specialist-celltrion-it-wouldn-t-say-no,news
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-marissa-tomei-partners-allergan-restasis-to-drive-dry-eye-awareness,news


In [61]:
grp=train.groupby('Tag')

In [10]:
#grp.groups

In [11]:
# for name,group in grp:
#     print(name)
#     print(group)

In [29]:
train.Tag.unique()

array(['news', 'clinicalTrials', 'conferences', 'profile', 'forum',
       'publication', 'thesis', 'guidelines', 'others'], dtype=object)

In [62]:
test_set=[]
train_set=[]
test_tag_set=[]
train_tag_set=[]
count=0
for unique_tags in train.Tag.unique():
    tag=grp.get_group(unique_tags)
    s=set(tag.Domain)
    count+=len(s)
    a=list(s)
    #shuffle(a)
    for i in range(len(a)):
        if (i+1)%3==0:
            test_set.append(a[i])
            test_tag_set.append(unique_tags)
        else:
            train_set.append(a[i])
            train_tag_set.append(unique_tags)

In [31]:
len(test_set)

1595

In [32]:
len(train_set)

3192

In [33]:
3192/1595

2.0012539184952978

In [34]:
len(train.Domain.unique())

3974

In [35]:
a=set(test_set)
b=set(train_set)
print('intersection',len(a.intersection(b)))
len(a.union(b))
#len(a)+len(b)

intersection 305


3974

#this verifies correct spliting 

In [67]:
len(train_tag_set)

3192

In [68]:
# now making train and val dataset of unique domains

train_domain=pd.DataFrame(train_set)

In [69]:
train_domain.columns=['Domain']
train_domain['Tag']=train_tag_set

In [70]:
train_domain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 2 columns):
Domain    3192 non-null object
Tag       3192 non-null object
dtypes: object(2)
memory usage: 50.0+ KB


In [71]:
train_data=pd.merge(train_domain,train,on=['Domain','Tag'],how='left')

In [72]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36631 entries, 0 to 36630
Data columns (total 4 columns):
Domain        36631 non-null object
Tag           36631 non-null object
Webpage_id    36631 non-null int64
Url           36631 non-null object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [73]:
train_data.head()

,Domain,Tag,Webpage_id,Url
0,ir.amagpharma.com,news,1497,http://ir.amagpharma.com/phoenix.zhtml?c=61596&p=irol-news&nyo=1
1,ir.amagpharma.com,news,1791,http://ir.amagpharma.com/phoenix.zhtml?c=61596&p=irol-news&nyo=0
2,ir.amagpharma.com,news,9622,http://ir.amagpharma.com/phoenix.zhtml?c=61596&p=irol-news&nyo=3
3,ir.amagpharma.com,news,9686,http://ir.amagpharma.com/corporate.rss?c=61596&Rule=Cat=news~subcat=ALL
4,ir.amagpharma.com,news,10026,http://ir.amagpharma.com/phoenix.zhtml?c=61596&p=irol-news&nyo=4


In [74]:
len(train_data.Webpage_id.unique())

36631

In [75]:
val_domain=pd.DataFrame(test_set)
val_domain.columns=['Domain']
val_domain['Tag']=test_tag_set

In [76]:
val_domain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 2 columns):
Domain    1595 non-null object
Tag       1595 non-null object
dtypes: object(2)
memory usage: 25.0+ KB


In [77]:
val_data=pd.merge(val_domain,train,on=['Domain','Tag'],how='left')

In [78]:
val_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 0 to 16815
Data columns (total 4 columns):
Domain        16816 non-null object
Tag           16816 non-null object
Webpage_id    16816 non-null int64
Url           16816 non-null object
dtypes: int64(1), object(3)
memory usage: 656.9+ KB


In [79]:
len(val_data.Webpage_id.unique())

16816

In [80]:
a=set(train_data.Webpage_id)
b=set(val_data.Webpage_id)
len(a.union(b))

53447

In [81]:
new_train=train_data.drop_duplicates(subset=['Webpage_id'], keep='first')

In [82]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36631 entries, 0 to 36630
Data columns (total 4 columns):
Domain        36631 non-null object
Tag           36631 non-null object
Webpage_id    36631 non-null int64
Url           36631 non-null object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [83]:
new_val=val_data.drop_duplicates(subset=['Webpage_id'], keep='first')

In [84]:
new_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 0 to 16815
Data columns (total 4 columns):
Domain        16816 non-null object
Tag           16816 non-null object
Webpage_id    16816 non-null int64
Url           16816 non-null object
dtypes: int64(1), object(3)
memory usage: 656.9+ KB


In [86]:
a=set(new_train.Webpage_id)
b=set(new_val.Webpage_id)
len(a.intersection(b))

0

In [87]:
len(new_train.Url.unique())

36464

In [88]:
#THIS CONFIRMS PROPER TRAIN AND VAL SPLIT

In [89]:
def train_val_split(train):
    grp=train.groupby('Tag')
    
    test_set=[]
    train_set=[]
    test_tag_set=[]
    train_tag_set=[]
    count=0
    for unique_tags in train.Tag.unique():
        tag=grp.get_group(unique_tags)
        s=set(tag.Domain)
        count+=len(s)
        a=list(s)
        #shuffle(a)
        for i in range(len(a)):
            if (i+1)%3==0:
                test_set.append(a[i])
                test_tag_set.append(unique_tags)
            else:
                train_set.append(a[i])
                train_tag_set.append(unique_tags)
    
    
    train_domain=pd.DataFrame(train_set)
    train_domain.columns=['Domain']
    train_domain['Tag']=train_tag_set
    train_data=pd.merge(train_domain,train,on=['Domain','Tag'],how='left')
    
    val_domain=pd.DataFrame(test_set)
    val_domain.columns=['Domain']
    val_domain['Tag']=test_tag_set
    val_data=pd.merge(val_domain,train,on=['Domain','Tag'],how='left')
    
    return(train_data,val_data)
    

In [90]:
train_data,val_data=train_val_split(train)

In [93]:
a=set(train_data.Webpage_id)
b=set(val_data.Webpage_id)
len(a.union(b))

53447

In [94]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36631 entries, 0 to 36630
Data columns (total 4 columns):
Domain        36631 non-null object
Tag           36631 non-null object
Webpage_id    36631 non-null int64
Url           36631 non-null object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [96]:
val_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16816 entries, 0 to 16815
Data columns (total 4 columns):
Domain        16816 non-null object
Tag           16816 non-null object
Webpage_id    16816 non-null int64
Url           16816 non-null object
dtypes: int64(1), object(3)
memory usage: 656.9+ KB


In [97]:
36631+16816

53447

In [273]:
import zipfile
import re
pattern=r'<title>.*</title>'

zf = zipfile.ZipFile('train.zip') 
df = pd.read_csv(zf.open('html_data.csv'),chunksize=1)

title=[]
chunksize = 1
for chunk in pd.read_csv(zf.open('html_data.csv'), chunksize=chunksize):
    a=chunk.Html
    idd=chunk.Webpage_id
    c=re.findall(pattern,a.iloc[0])
    if len(c)!=0:
        title.append([idd.iloc[0],c[0][7:-8]])
    else:
        title.append([idd.iloc[0],' '])

In [282]:
title_data=pd.DataFrame(title)
title_data.columns=['Webpage_id','Html']

In [284]:
title_data.to_csv('title_data.csv',index=False)


In [289]:
new_data=pd.merge(train,title_data,on=['Webpage_id'],how='left')

In [293]:
a=new_data[new_data.Webpage_id==1].Html

In [296]:
a.iloc[0]

'Tecfidera, Gilenya and Aubagio&#039;s 3-way battle for MS share is about to heat up | FiercePharma'

In [302]:
titles=[]
for e in new_data.Html:    
    e = ''.join([i for i in e if not i.isdigit()])
    f=''
    for i in e:
        if i.isalpha() or i==' ':
            #print(i)
            f+=i
        else:
            f+=' '
    f = re.sub(' +',' ',f) # replace series of spaces with single space
    titles.append(f)

In [303]:
len(titles)

53447

In [305]:
titles[:10]

['Tecfidera Gilenya and Aubagio s way battle for MS share is about to heat up FiercePharma',
 'Novo equipped to weather the storm in the U S diabetes market CEO says FiercePharma',
 'Another exec departs troubled Endo and this time it s for another drugmaker FiercePharma',
 'Would Teva buy Korea s Celltrion to beef up in biosimilars It wouldn t say no FiercePharma',
 'Restasis maker Allergan recruits actress Marisa Tomei to drive dry eye awareness FiercePharma',
 'NICE backs less effective Otezla to give psoriatic arthritis patients a pill option FiercePharma',
 'Mylan takes on punching bag role again as Clinton aims to shame bad actors FiercePharma',
 'Failed Eylea combo trial throws a wrench in Regeneron s expansion effort FiercePharma',
 'Should GSK investors be disgruntled about its insider CEO pick If they want big changes yes FiercePharma',
 'A silver lining to the drug pricing uproar For generic competition yes GPhA chief says FiercePharma']

In [306]:
new_data_test=pd.merge(test,title_data,on=['Webpage_id'],how='left')

In [307]:
new_data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25787 entries, 0 to 25786
Data columns (total 4 columns):
Webpage_id    25787 non-null int64
Domain        25787 non-null object
Url           25787 non-null object
Html          25787 non-null object
dtypes: int64(1), object(3)
memory usage: 1007.3+ KB
